In [1]:
import xarray as xr
import numpy as np
import pandas as pd
#import rioxarray as rxr
#import datetime as dt
#from scipy.stats import norm
#import tools as utils
import warnings
warnings.filterwarnings('ignore')

Load the netCDF files from ERA5

In [5]:
t2m_raw = xr.open_dataset('/Volumes/Ryan_SSD/MAST638_Data/ERA5/t2m_2003_2023.nc',decode_coords="all")
# sst_raw = xr.open_dataset(utils.file_names['sstfile'],decode_coords='all')
# solar_raw = xr.open_dataset(utils.file_names['solarfile'],decode_coords='all')
# wind_raw = xr.open_dataset(utils.file_names['windfile'],decode_coords='all')
# ice_raw = xr.open_dataset(utils.file_names['seaicefile'],decode_coords='all')
# rad_raw = xr.open_dataset(utils.file_names['radfile'],decode_coords='all')
# rad2_raw = xr.open_dataset(utils.file_names['radfile_long'],decode_coords='all')
# cloud_raw = xr.open_dataset(utils.file_names['cloudfile'],decode_coords='all')
# rh_raw = xr.open_dataset(utils.file_names['rhfile'],decode_coords='all')

In [7]:
t2m_raw

<xarray.Dataset>
Dimensions:    (longitude: 29, latitude: 41, time: 29220)
Coordinates:
  * longitude  (longitude) float32 139.0 139.2 139.5 139.8 ... 145.5 145.8 146.0
  * latitude   (latitude) float32 -60.0 -60.25 -60.5 ... -69.5 -69.75 -70.0
  * time       (time) datetime64[ns] 2003-01-01 ... 2022-12-31T18:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-12-05 15:42:24 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [8]:
# drop all the Feb 29th in leap years

t2m_raw = t2m_raw.sel(time=~((t2m_raw.time.dt.month == 2) & (t2m_raw.time.dt.day == 29)))

# sst_raw     = sst_raw.sel(time=~((sst_raw.time.dt.month == 2) & (sst_raw.time.dt.day == 29)))

# solar_raw   = solar_raw.sel(time=~((solar_raw.time.dt.month == 2) & (solar_raw.time.dt.day == 29)))

# wind_raw    = wind_raw.sel(time=~((wind_raw.time.dt.month == 2) & (wind_raw.time.dt.day == 29)))

# ice_raw     = ice_raw.sel(time=~((ice_raw.time.dt.month == 2) & (ice_raw.time.dt.day == 29)))

# rad_raw     = rad_raw.sel(time=~((rad_raw.time.dt.month == 2) & (rad_raw.time.dt.day == 29)))

# rad2_raw    = rad2_raw.sel(time=~((rad2_raw.time.dt.month == 2) & (rad2_raw.time.dt.day == 29)))

# cloud_raw   = cloud_raw.sel(time=~((cloud_raw.time.dt.month == 2) & (cloud_raw.time.dt.day == 29)))

# rh_raw      = rh_raw.sel(time=~((rh_raw.time.dt.month == 2) & (rh_raw.time.dt.day == 29)))


In [9]:
t2m_raw

<xarray.Dataset>
Dimensions:    (longitude: 29, latitude: 41, time: 29200)
Coordinates:
  * longitude  (longitude) float32 139.0 139.2 139.5 139.8 ... 145.5 145.8 146.0
  * latitude   (latitude) float32 -60.0 -60.25 -60.5 ... -69.5 -69.75 -70.0
  * time       (time) datetime64[ns] 2003-01-01 ... 2022-12-31T18:00:00
Data variables:
    t2m        (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2023-12-05 15:42:24 GMT by grib_to_netcdf-2.25.1: /opt/ecmw...

In [5]:
# calculate the wind magnitude and wind direction

wind_raw['magnitude'] = np.sqrt(np.square(wind_raw['u10']) + np.square(wind_raw['v10']))
wind_raw['direction'] = np.rad2deg(np.arctan2(wind_raw['v10'],wind_raw['u10']))


In [10]:
weights = np.cos(np.deg2rad(t2m_raw.latitude))
weights.name='weights'

In [5]:
ice_reproj = ice_raw.rio.reproject('epsg:4326')

In [ ]:
# get the DDU area form the sea ice file and pull out only the mid concentation
ice_area_mask = ice_reproj['tc_mid'].sel(y=slice(-60,-70),
                                           x=slice(139,146))
    
    ddu_xd = xd.where(mask_ddu)
    
            ddu_df = ddu_xd[varname].where((ice_reproj[varname] <= 100.)).weighted(iweights).mean(dim=['x','y']).to_dataframe()

In [6]:
# weight the cells for the ice data 

iweights = np.cos(np.deg2rad(ice_reproj.y))
iweights.name='weights'

In [11]:
t2m_1d = t2m_raw.weighted(weights).mean(dim=['longitude','latitude']).to_dataframe()

In [14]:
t2m_weekly = t2m_1d.resample('W').mean()

Full Scale Vars Processing

In [ ]:
ice_master_ddu_df = utils.combineDFS(iddu_list)
ice_master_mawson_df = utils.combineDFS(imawson_list)
ice_master_somov_df = utils.combineDFS(isomov_list)
ice_master_study_df = utils.combineDFS(istudy_list)

In [ ]:
# merge atmo and ice data (auto cuts to available dates, do how='left' to not)

all_study_area = pd.merge(master_study_df,ice_master_study_df,left_index=True,right_index=True,how='left')
all_ddu_area = pd.merge(master_ddu_df,ice_master_ddu_df,left_index=True,right_index=True,how='left')
all_mawson_area = pd.merge(master_mawson_df,ice_master_mawson_df,left_index=True,right_index=True,how='left')
all_somov_area = pd.merge(master_somov_df,ice_master_somov_df,left_index=True,right_index=True,how='left')

In [ ]:
study_file = '/Users/ryaneagan/GaugeData/era_5_data/full_area/study.xlsx'
ddu_file = '/Users/ryaneagan/GaugeData/era_5_data/full_area/ddu.xlsx'
mawson_file = '/Users/ryaneagan/GaugeData/era_5_data/full_area/mawson.xlsx'
somov_file = '/Users/ryaneagan/GaugeData/era_5_data/full_area/somov.xlsx'

In [ ]:
all_study_area.to_excel(study_file)
all_ddu_area.to_excel(ddu_file)
all_mawson_area.to_excel(mawson_file)
all_somov_area.to_excel(somov_file)